### Imports

In [ ]:
import re
import string
import math
import itertools
from collections import defaultdict, Counter
from operator import itemgetter

### Alphabet

In [ ]:
ALPHABET = string.ascii_uppercase
print(f"[*] Alphabet:\n{'.'*len(ALPHABET)}\n{ALPHABET}\n{'*'*len(ALPHABET)}")

### Utils

In [ ]:
def _assert_alphabet(text):
    assert all(c in ALPHABET for c in text)

def alphabet_diff(a, b):
    """Compute (a - b) inside alphabet."""
    return ALPHABET[(ALPHABET.index(a) - ALPHABET.index(b)) % len(ALPHABET)]

def alphabet_sum(a, b):
    """Compute (a + b) inside alphabet."""
    return ALPHABET[(ALPHABET.index(a) + ALPHABET.index(b)) % len(ALPHABET)]

def text2message(text):
    return "".join(c for c in text.upper() if c in ALPHABET)

def print_enumerated(iterable, s):
    print(f'[*] {s}:')
    for i, item in enumerate(iterable, start=1):
        t = str(item)
        if len(t) > 100:
            print(f'{i: >2}: {t[:80]}...')
        else:
            print(f'{i: >2}: {t}')

### Vigener Algorithm

In [4]:
def _vigener_encrypt(message, key):
    """
    Encrypt message using `Vigener algorithm`_.
    
    Args:
        message (str): Message to encrypt.
        key (str): Secret key.
    
    Returns:
        str: Encrypted message.
    
    .. _Vigener algorithm:
        https://en.wikipedia.org/wiki/Vigenère_cipher
    """
    
    _assert_alphabet(message)
    _assert_alphabet(key)
    
    key_len = len(key)
    ciphertext = ''
    
    for i, mi in enumerate(message):
        # Note: mi + ki = ci
        # Note: ki = K[ki mod len(K)]
        ki = key[i % key_len]
        ci = alphabet_sum(mi, ki)
        ciphertext += ci
    
    return ciphertext

def vigener_encrypt(text, key, *, preserve_whitespace=False):
    if preserve_whitespace:
        wss = [m.start() for m in re.finditer(' ', text)]
        ciphertext = vigener_encrypt(text.replace(' ', ''), key)
        l = list(ciphertext)
        for p in wss:
            l.insert(p, ' ')
        return ''.join(l)
    else:
        return _vigener_encrypt(text2message(text), key.upper())

### Test Vigener encryption

In [5]:
# Example from https://en.wikipedia.org/wiki/Vigenère_cipher
text = "ATTACK AT DAWN"
key = "LEMON"
answer = "LXFOPV EF RNHR"
ciphertext = vigener_encrypt(text, key, preserve_whitespace=True)

print(f"[*] Text: {text}")
print(f"[*] Key: {key}")
print(f"[+] Ciphertext: {ciphertext}")

assert ciphertext == answer, "Encryption failed"

[*] Text: ATTACK AT DAWN
[*] Key: LEMON
[+] Ciphertext: LXFOPV EF RNHR


### Encrypt some text

In [6]:
# Texts were generated by http://metaphorpsum.com/
# input_filename = "input-1-20.txt"
# input_filename = "input-1-30.txt"
input_filename = "input-1-50.txt"
# input_filename = "input-10-50.txt"
# input_filename = "input-4-50.txt"

In [7]:
with open(input_filename) as f:
    text = f.read()

message = text2message(text)
key = "MOUSE"
ciphertext = vigener_encrypt(text, key)

# with open("output.txt", "w") as f:
#     f.write(ciphertext)

print(f"[*] Text: {text[:70]}...")
print(f"[*] Message:    {message[:70]}...")
print(f"[+] Ciphertext: {ciphertext[:70]}...")

[*] Text: Some posit the croaky wedge to be less than funded. One cannot separat...
[*] Message:    SOMEPOSITTHECROAKYWEDGETOBELESSTHANFUNDEDONECANNOTSEPARATEALGERIASFROM...
[+] Ciphertext: ECGWTAGCLXTSWJSMYSOIPUYLSNSFWWEHBSRRIHVIPCHWGMBHGXESJSVMHYSPSSLAEETLGQ...


### Kasiski Examination

In [8]:
def kasiski_examination(ciphertext, seq_len, max_key_len, *, verbose=False):
    """
    Find the most probable key length using `Kasiski examination`_.
    
    Args:
        ciphertext (str): Vigener-ciphered text.
        seq_len (int): Length of analyzed substrings.
        max_key_len (int): Upper bound for the key length.
        verbose (bool): Be verbose.
    
    Returns:
        int: The most probable key length.
    
    .. _Kasiski examination:
        https://en.wikipedia.org/wiki/Kasiski_examination
    """
    
    _assert_alphabet(ciphertext)

    # Find positions of each substring of length `seq_len`
    seq_positions = defaultdict(list)  # {seq: [pos]}
    for i in range(len(ciphertext) - seq_len):
        seq_positions[ciphertext[i : i + seq_len]].append(i)
    
    # Drop non-repeated sequences
    seq_positions = {
        seq: positions
        for seq, positions in seq_positions.items()
        if len(positions) >= 2
    }
    assert len(seq_positions) > 0, f"No repeated sequences of length {seq_len}"
    
    if verbose:
        print("[*] Positions:")
        for seq, positions in seq_positions.items():
            print(f"    {seq}: {positions}")

    # Calculte spacings between subsequent positions for each sequence
    seq_spacings = defaultdict(list)  # {seq: [space]}
    for seq, positions in seq_positions.items():
        for a, b in zip(positions, positions[1:]):
            seq_spacings[seq].append(b - a)
    
    if verbose:
        print("[*] Spacings:")
        for seq, spacings in seq_spacings.items():
            print(f"    {seq}: {spacings}")
    
    # Count factors (<=max_key_len) of all spacings
    factor_count = Counter()
    for spacings in seq_spacings.values():
        for space in spacings:
            for f in range(2, min(max_key_len, int(math.sqrt(space)) + 1)):
                if space % f == 0:
                    factor_count[f] += 1
    
    if verbose:
        print("[*] Factor counts:")
        for f, count in factor_count.items():
            print(f"    {f}: {count}")
    
    # Find the most probable key length
    key_len = factor_count.most_common()[0][0]
    
    if verbose:
        print(f"[+] The most probable key length: {key_len}")
    
    return key_len

### Perform Kasiski Examination

In [9]:
# probable_key_len = kasiski_examination(ciphertext, 10, 10, verbose=True)

In [10]:
probable_key_len = kasiski_examination(ciphertext, 5, 10)
probable_key_len

5

### Frequence Analysis

In [11]:
def _freq(text):
    return Counter(text)

def _blocks(ciphertext, key_len):
    return [''.join(ciphertext[shift + i]
                    for i in range(0, len(ciphertext) - shift, key_len))
            for shift in range(key_len)]

def frequency_analysis(ciphertext, key_len, text_freq, *, verbose=False):
    """
    Find the most probable key using `frequency analysis`_.
    
    Args:
        ciphertext (str): Vigener-ciphered text.
        key_len (int): Key length.
        text_freq (collections.Counter): Estimated table of letter frequencies.
    
    Returns:
        str: The most probable key found by `frequency analysis`_.
    
    .. _frequency analysis:
        https://en.wikipedia.org/wiki/Frequency_analysis
    """
    
    _assert_alphabet(ciphertext)
    
    blocks = _blocks(ciphertext, key_len)
    block_freqs = [_freq(block) for block in blocks]
    most_common_letter = text_freq.most_common()[0][0]
    
    if verbose:
        print_enumerated(blocks, 'Blocks')
        print("[*] Block frequencies (most common):")
        for i, block_freq in enumerate(block_freqs, start=1):
            print(f"{i: >2}: {block_freq.most_common(7)}")
        print(f"[*] Most common letter in original text: '{most_common_letter}'")
    
    # Find the most probable key
    key = ''
    for i, block in enumerate(blocks):
        # Note: mi + ki = ci, then ki = ci - mi
        ci = block_freqs[i].most_common()[0][0]
        ki = alphabet_diff(ci, most_common_letter)
        key += ki
    
    if verbose:
        print(f"[+] Most probable key: '{key}'")
    
    return key

In [12]:
text_freq = _freq(message)
text_freq.most_common(7)

[('E', 316),
 ('A', 266),
 ('T', 263),
 ('S', 203),
 ('I', 192),
 ('N', 184),
 ('O', 176)]

In [13]:
probable_key = frequency_analysis(ciphertext, probable_key_len, text_freq, verbose=True)

[*] Blocks:
 1: EATMPNERPMEMSENXUEBFMEONEPQRUODFTIAOMDOMPAUFAFTODZFUEMMAGAYMQIQYEIQRIYFQYUGUMAEZ...
 2: CGSYUSHICBSHSTFSBGCVGGSSHORWGIHSOFBCFOHBSAHVIVSSCSCZYZGWQBSRFODSOGBHOSIOSHBBJPGI...
 3: GCWSYFBHHHJYLLOMXIMYYXLFBJNVUHOFNINHSMUCUYBYAUIHGCYUYFJXUZXCYSUHVSWBMQHLJNMYYYNH...
 4: WLJOLWSVWGSSAGKKGEAUDSLWSHZJHXFQAFZLSLFSVKALZLTWTKFTLASWLJAXFSJLDXZAKZUKGZLVFDZU...
 5: TXSISWRIGXVPEQLAAIXIIRSWRPIIZSEXWKIVHMDWGAXLXXWXSEEEFWVHMERJXHXMSVMWSEPSWIEVXIEV...
[*] Block frequencies (most common):
 1: [('Q', 61), ('F', 57), ('M', 53), ('A', 44), ('E', 39), ('Z', 39), ('U', 38)]
 2: [('S', 65), ('H', 55), ('O', 48), ('C', 42), ('B', 40), ('G', 35), ('W', 34)]
 3: [('Y', 66), ('U', 57), ('N', 45), ('M', 40), ('H', 39), ('L', 38), ('C', 36)]
 4: [('W', 67), ('S', 53), ('L', 51), ('K', 46), ('A', 45), ('F', 35), ('J', 31)]
 5: [('I', 57), ('X', 55), ('E', 55), ('W', 43), ('M', 39), ('V', 36), ('S', 34)]
[*] Most common letter in original text: 'E'
[+] Most probable key: 'MOUSE'


In [14]:
probable_key

'MOUSE'

### Decrypt message

In [15]:
def vigener_decrypt(ciphertext, key):
    """
    Decrypt Vigener-ciphered text.
    
    Args:
        ciphertext (str): Vigener-ciphered text.
        key (int): Secret key.
    
    Returns:
        str: Decrypted message.
    """
    
    key_len = len(key)
    message = ''
    
    for i, ci in enumerate(ciphertext):
        # Note: mi + ki = ci, then mi = ci - ki
        ki = key[i % key_len]
        mi = alphabet_diff(ci, ki)
        message += mi
    
    return message

In [16]:
decrypted_message = vigener_decrypt(ciphertext, probable_key)

print(f"[*] Text:       {text[:70]}...")
print(f"[*] Message:    {message[:70]}...")
print(f"[*] Ciphertext: {ciphertext[:70]}...")
print(f"[+] Decrypted:  {decrypted_message[:70]}...")

assert decrypted_message == message, "Decryption failed"

[*] Text:       Some posit the croaky wedge to be less than funded. One cannot separat...
[*] Message:    SOMEPOSITTHECROAKYWEDGETOBELESSTHANFUNDEDONECANNOTSEPARATEALGERIASFROM...
[*] Ciphertext: ECGWTAGCLXTSWJSMYSOIPUYLSNSFWWEHBSRRIHVIPCHWGMBHGXESJSVMHYSPSSLAEETLGQ...
[+] Decrypted:  SOMEPOSITTHECROAKYWEDGETOBELESSTHANFUNDEDONECANNOTSEPARATEALGERIASFROM...
